In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [5]:
CLIENT_ID = 'ZJLDAZ3WP20OIO1SE3C2LEDFQLKDPAWEXFVWN0PVQ1HD5AQJ' # your Foursquare ID
CLIENT_SECRET = 'EUSQQXHATVYTT3GQ0OAK4550T0ILKHMJA41Z2SG1JZZAWN2F' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 10000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZJLDAZ3WP20OIO1SE3C2LEDFQLKDPAWEXFVWN0PVQ1HD5AQJ
CLIENT_SECRET:EUSQQXHATVYTT3GQ0OAK4550T0ILKHMJA41Z2SG1JZZAWN2F


In [6]:
address = 'Belleville, Paris, France'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

48.8717265 2.385085


In [7]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=ZJLDAZ3WP20OIO1SE3C2LEDFQLKDPAWEXFVWN0PVQ1HD5AQJ&client_secret=EUSQQXHATVYTT3GQ0OAK4550T0ILKHMJA41Z2SG1JZZAWN2F&ll=48.8717265,2.385085&v=20180604&limit=100'

In [8]:
results = requests.get(url).json()

In [9]:
# assign relevant part of JSON to venues
# tranform venues into a dataframe
venue_df = json_normalize([item['venue'] for group in results['response']['groups'] for item in group['items']])

In [10]:
venue_df['categories'] = [category[0]['name'] for category in venue_df['categories']]
venue_df['location.formattedAddress'] = [x[0] for x in venue_df['location.formattedAddress'].values]
venue_df.head()

,categories,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,photos.count,photos.groups,venuePage.id
0,Park,4adcda0af964a520253421e3,47 rue des Couronnes,FR,Paris,France,NaN,30,47 rue des Couronnes,"[{'label': 'display', 'lat': 48.87145729102217...",48.871457,2.385014,NaN,75020,Île-de-France,Parc de Belleville,0,[],NaN
1,French Restaurant,4b75e3f0f964a520742c2ee3,12 rue des Envierges,FR,Paris,France,NaN,85,12 rue des Envierges,"[{'label': 'display', 'lat': 48.87166048715102...",48.871660,2.386255,NaN,75020,Île-de-France,Le Vieux Belleville,0,[],NaN
2,Pizza Place,5471f9b3498e6e2065dc3ad4,356 rue des Pyrénées,FR,Paris,France,NaN,261,356 rue des Pyrénées,"[{'label': 'display', 'lat': 48.87391757999977...",48.873918,2.386359,NaN,75020,Île-de-France,Il Posto,0,[],NaN
3,French Restaurant,4adcda14f964a5203c3721e3,3 rue Jouye-Rouve,FR,Paris,France,NaN,228,3 rue Jouye-Rouve,"[{'label': 'display', 'lat': 48.87309786827826...",48.873098,2.382767,NaN,75020,Île-de-France,Le Baratin,0,[],NaN
4,French Restaurant,54dd4f8f498e62876939b098,101 rue des Couronnes,FR,Paris,France,NaN,321,101 rue des Couronnes,"[{'label': 'display', 'lat': 48.8722765406218,...",48.872277,2.389393,NaN,75020,Île-de-France,Le Jourdain,0,[],NaN


In [11]:
venue_df.drop(columns={"location.cc", "location.labeledLatLngs", "location.neighborhood","location.address", "photos.count", "photos.groups", "venuePage.id", "location.crossStreet"}, inplace = True)

In [12]:
venue_df.columns = ['categories', 'id', 'city', 'country', 'distance', 'address', 'lat', 'lng', 'postalCode', 'state', 'name']

In [13]:
venue_df.dropna(subset = ["postalCode"], axis=0, inplace = True)

In [14]:
venue_df.head()

,categories,id,city,country,distance,address,lat,lng,postalCode,state,name
0,Park,4adcda0af964a520253421e3,Paris,France,30,47 rue des Couronnes,48.871457,2.385014,75020,Île-de-France,Parc de Belleville
1,French Restaurant,4b75e3f0f964a520742c2ee3,Paris,France,85,12 rue des Envierges,48.871660,2.386255,75020,Île-de-France,Le Vieux Belleville
2,Pizza Place,5471f9b3498e6e2065dc3ad4,Paris,France,261,356 rue des Pyrénées,48.873918,2.386359,75020,Île-de-France,Il Posto
3,French Restaurant,4adcda14f964a5203c3721e3,Paris,France,228,3 rue Jouye-Rouve,48.873098,2.382767,75020,Île-de-France,Le Baratin
4,French Restaurant,54dd4f8f498e62876939b098,Paris,France,321,101 rue des Couronnes,48.872277,2.389393,75020,Île-de-France,Le Jourdain


In [15]:
venue_df['categories'].value_counts(normalize=True).head()

Bar                   0.195876
French Restaurant     0.092784
Italian Restaurant    0.051546
Café                  0.051546
Restaurant            0.030928
Name: categories, dtype: float64

In [16]:
venue_df['categories_id'] = pd.factorize(venue_df.categories)[0]
venue_df.head()

,categories,id,city,country,distance,address,lat,lng,postalCode,state,name,categories_id
0,Park,4adcda0af964a520253421e3,Paris,France,30,47 rue des Couronnes,48.871457,2.385014,75020,Île-de-France,Parc de Belleville,0
1,French Restaurant,4b75e3f0f964a520742c2ee3,Paris,France,85,12 rue des Envierges,48.871660,2.386255,75020,Île-de-France,Le Vieux Belleville,1
2,Pizza Place,5471f9b3498e6e2065dc3ad4,Paris,France,261,356 rue des Pyrénées,48.873918,2.386359,75020,Île-de-France,Il Posto,2
3,French Restaurant,4adcda14f964a5203c3721e3,Paris,France,228,3 rue Jouye-Rouve,48.873098,2.382767,75020,Île-de-France,Le Baratin,1
4,French Restaurant,54dd4f8f498e62876939b098,Paris,France,321,101 rue des Couronnes,48.872277,2.389393,75020,Île-de-France,Le Jourdain,1


In [17]:

venues_map = folium.Map(location=[latitude, longitude], zoom_start=100) # generate map centred around the Conrad Hotel
colors_array = cm.rainbow(np.linspace(0, 1, len(set(venue_df.categories_id))))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Belle Ville',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label, factory_id in zip(venue_df.lat, venue_df.lng, venue_df.categories, venue_df.categories_id):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color=rainbow[factory_id],
        popup=label,
        fill = True,
        fill_color=rainbow[factory_id],
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map


In [18]:
# one hot encoding
venue_df_onehot = pd.get_dummies(venue_df[['categories']], prefix="", prefix_sep="")


# add distancecolumn back to dataframe
venue_df_onehot['distance'] = venue_df['distance'] 

# move neighborhood column to the first column
fixed_columns = [venue_df_onehot.columns[-1]] + list(venue_df_onehot.columns[:-1])
venue_df_onehot = venue_df_onehot[fixed_columns]

venue_df_onehot.head()

,distance,African Restaurant,Asian Restaurant,Bakery,Bar,Beer Bar,Beer Store,Bistro,Bookstore,Brewery,...,Speakeasy,Street Food Gathering,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,85,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,261,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,228,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,321,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
venue_df.drop(columns = {'categories_id'}, inplace = True)
venue_df = venue_df.reset_index(drop = True)
venue_df_grouped = venue_df_onehot.groupby('distance').mean().reset_index()

KeyError: "['categories_id'] not found in axis"

In [ ]:
num_top_venues = 5

for hood in venue_df_grouped['distance']:
    print("----"+str(hood)+"----")
    temp = venue_df_grouped[venue_df_grouped['distance'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head())
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
# set number of clusters
kclusters = 5

venue_grouped_clustering = venue_df_grouped.drop('distance', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venue_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
venue_df_grouped = venue_df_grouped.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

venue_df_grouped.head() # check the last columns!

In [ ]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters